<a href="https://colab.research.google.com/github/Netdrum/MARIA/blob/main/FineTuneWhisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

In [ ]:
!pip install datasets
!pip install librosa
!pip install soundfile
!pip install evaluate
!pip install jiwer

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

ds = load_dataset("Netdrum/IRCG_VHF")

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperProcessor, WhisperForConditionalGeneration
import librosa # Import librosa

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Assuming 'audio_file_path' contains the path to your audio file:
audio_input, sample_rate = librosa.load("/content/drive/MyDrive/Dataset")  # Load your audio data using librosa
# Now you need to process the audio data using the feature extractor or the processor
input_features = processor(
    audio_input,
    sampling_rate=sample_rate,
    return_tensors="pt"
).input_features  # Process the audio data to obtain input features

model_outputs = model.generate(
    input_features,
    chunk_length_s=30  # Process 30-second chunks
)

In [7]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="English", task="transcribe")
from transformers import WhisperProcessor


In [ ]:
import numpy as np
ds = load_dataset("Netdrum/IRCG_VHF")
input_audio = ds["train"][0]["audio"]["array"]
input_str = str(input_audio.tolist())
labels =tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels)
decoded_str = tokenizer.convert_ids_to_tokens(labels)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {iniput_str == decoded_with_special}")

In [ ]:
ds["train"].features

In [ ]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="English", task="transcribe")

In [ ]:
print(ds["train"][0])

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    #compute log-mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(au

    #compute input lenght of audio sample in seconds
    if features is not None and hasattr(features, "input_features") else
      batch["input_features"] = features.input_features[0]
        print("Warning: features is None or input_features is missing for this example.")
        batch["input_features"] = None  # Or raise an exception, or skip this example

    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    text_column = "transcription"  # Replace with your actual column name if needed
    if text_column in batch:
        batch["labels"] = processor.tokenizer(batch[text_column]).input_ids
    else:
        # Handle the case where the expected column is missing
        # You might want to raise an exception or skip this example
        raise KeyError(f"Column '{text_column}' not found in the dataset.")

    # encode target text to label ids
    batch["labels"] = processor.tokenizer(batch["transcription"]).input_ids
    return batch

In [ ]:
ds = ds.map(prepare_dataset, remove_columns=list(next(iter(ds.values())).features)).with_formate("torch")

In [ ]:
ds["train"] = ds["train"].shuffle(
    seed=42,
    buffer_size=10,
)

In [ ]:
max_input_length = 30.0

def is_audio_in_length_range(length):
    return length < max_input_length

In [ ]:
ds["train"] = ds["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model.generation_config.language = "english"
model.generation_config.task = "transcribe"

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeqWithPadding:
  processor: Any
  decoder_start_token_id: int


  def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
    # split inputs and labels since they have to be of different lengths and need
    # different padding methods
    input_features = [{"input_features": feature["labels"]} for feature in features]
    batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

    label_features = [{"input_ids": feature["labels"]} for feature in features]
    # pad the labels to max length
    labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

    labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

    batch["labels"] = labels

    return batch



In [ ]:
data_collator = DataCollatorSpeechSeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./Netdrum/IRCG_VHF",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)